PROSES PEMBERIAN LABEL MENGGUNAKAN RUMUS RFM

In [ ]:
import pandas as pd

df = pd.read_excel('pelangganSales.xlsx')

df.head()

,TransactionID,CustomerID,CustomerName,TransactionDate,JumlahBarang,JumlahHarga
0,1,716,David Davenport,2024-03-19,67,792514
1,2,465,Jose Allen,2024-08-21,59,46588
2,3,827,Heather Brown,2023-03-24,2,308322
3,4,925,Danielle Murray,2024-05-07,6,768217
4,5,454,Cynthia Vang,2023-06-17,96,280247


Cetak tipe data dan menampilkan jumlah baris yang mengandung nilai null per kolom

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   TransactionID    100000 non-null  int64 
 1   CustomerID       100000 non-null  int64 
 2   CustomerName     100000 non-null  object
 3   TransactionDate  100000 non-null  object
 4   JumlahBarang     100000 non-null  int64 
 5   JumlahHarga      100000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.6+ MB


In [ ]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format='%Y-%m-%d')

Menambahkan kolom yang berisi nilai Recency, Frequency, Monetery (skala 1 - 4)

Skala Recency : semakin besar nilainya semakin baik

In [ ]:
snapshot_date = df['TransactionDate'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'TransactionDate': lambda x: (snapshot_date - x.max()).days,
    'JumlahBarang': 'count',
    'JumlahHarga': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']
rfm.head()

,CustomerID,Recency,Frequency,Monetary
0,1,4,95,47091754
1,2,11,109,56723859
2,3,6,101,49087359
3,4,2,110,51894173
4,5,4,112,53705292


In [ ]:
rfm['R_Score'] = pd.qcut(rfm['Recency'], 4, labels=[1, 2, 3, 4]).astype(int)
rfm['F_Score'] = pd.qcut(rfm['Frequency'], 4, labels=[1, 2, 3, 4]).astype(int)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4]).astype(int)

In [ ]:
rfm['RFM'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

In [ ]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM
0,1,4,95,47091754,2,2,2,222
1,2,11,109,56723859,3,4,4,344
2,3,6,101,49087359,2,3,2,232
3,4,2,110,51894173,1,4,3,143
4,5,4,112,53705292,2,4,3,243


In [ ]:
def rfm_category(rfm_code):
    if '4' in rfm_code:
        return 'Platinum'
    elif '3' in rfm_code:
        return 'Gold'
    elif '2' in rfm_code:
        return 'Silver'
    else:
        return 'Bronze'


In [ ]:
rfm['Category'] = rfm['RFM'].apply(rfm_category)

In [ ]:
rfm.head()

,CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM,Category
0,1,4,95,47091754,2,2,2,222,Silver
1,2,11,109,56723859,3,4,4,344,Platinum
2,3,6,101,49087359,2,3,2,232,Gold
3,4,2,110,51894173,1,4,3,143,Platinum
4,5,4,112,53705292,2,4,3,243,Platinum


In [ ]:
data = pd.merge(df, rfm[['CustomerID', 'Category']], on='CustomerID', how='left')

In [ ]:
data.info()
rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   TransactionID    100000 non-null  int64         
 1   CustomerID       100000 non-null  int64         
 2   CustomerName     100000 non-null  object        
 3   TransactionDate  100000 non-null  datetime64[ns]
 4   JumlahBarang     100000 non-null  int64         
 5   JumlahHarga      100000 non-null  int64         
 6   Category         100000 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 5.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  1000 non-null   int64 
 1   Recency     1000 non-null   int64 
 2   Frequency   1000 non-null   int64 
 3   Monetary    1000 non-null 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
X = rfm[['Recency','Frequency','Monetary']]
y = rfm['Category']

In [ ]:
le = LabelEncoder()
y_enc = le.fit_transform(y)

In [ ]:
X_train, X_test, y_train_enc, y_test_enc = train_test_split(X, y_enc, stratify=y_enc, test_size=0.2, random_state = 42)


In [ ]:
model = XGBClassifier(
    objective='multi:softmax',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
model.fit(X_train, y_train_enc)
y_pred = model.predict(X_test)

print("Akurasi :", accuracy_score(y_test_enc, y_pred))
print("Classification Report:")
print(classification_report(y_test_enc, y_pred))


Akurasi : 0.985
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.97      1.00      0.98        60
           2       1.00      0.99      0.99        96
           3       0.97      0.97      0.97        35

    accuracy                           0.98       200
   macro avg       0.98      0.96      0.97       200
weighted avg       0.99      0.98      0.98       200



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:50:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
scores = cross_val_score(model, X, y_enc, cv=5)

# Final model untuk prediksi nyata (100% data)
model.fit(X, y_enc)

print("Akurasi tiap fold (CV):", scores)
print("Rata-rata akurasi CV:", scores.mean())

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:50:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:50:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:50:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:50:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [00:50:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

Akurasi tiap fold (CV): [1.   1.   1.   0.99 1.  ]
Rata-rata akurasi CV: 0.998
